In [84]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.layers import LSTM,Reshape,Layer
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from tensorflow.keras.layers import Conv2D,Flatten,ConvLSTM2D,Conv1D
import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.backend import argmax
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.utils import to_categorical

In [85]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataY.append(dataset[i + look_back, 0]) 
        dataX.append(a)
    return numpy.array(dataX), numpy.array(dataY)

In [86]:
def create_resultset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        dataY.append(dataset.iloc[i + look_back, 0]) 
    return  numpy.array(dataY)

In [87]:
import datasets 
from datasets import load_ypred, load_usps, load_mrec

Y=load_ypred()

#Y.shape
Ysize=len(numpy.bincount(Y.iloc[:,0]))


MM (4030, 1)
YPRED: (4030, 1)


In [88]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
print('len',len(Y))
ZZ=pad_sequences(numpy.array(Y),maxlen=10)
ZZ

len 4030


array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 9],
       [0, 0, 0, ..., 0, 0, 2],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 9],
       [0, 0, 0, ..., 0, 0, 3]], dtype=int32)

In [89]:
ww=numpy.array([[1],[2],[2]])
ww.shape

(3, 1)

In [90]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 0.999))
YM = scaler.fit_transform(Y)
#YM=Y.loc[:,:]
#YM

numpy.bincount(Y.iloc[:,0])

array([219, 491, 466, 208, 378, 268, 298,  82,  64, 448, 128, 240, 152,
       266, 196, 126])

In [91]:
look_back = 20
trainX0, _ = create_dataset(YM, look_back)
trainY =create_resultset(Y,look_back)
Ysize=16


In [92]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX0, (trainX0.shape[0], trainX0.shape[1],1))
trainX0.shape

#testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

(4009, 20)

In [93]:
trainYO = to_categorical(trainY, num_classes=Ysize)
trainYO.shape

(4009, 16)

In [123]:
class MyReshape(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyReshape, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        print('buikd',input_shape)
        self.w = tf.Variable(initial_value=tf.ones((1,20)),
                             trainable=False)
      
        super(MyReshape, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        print('call',x)
        input_shape=x.shape
        print('call inp',input_shape)
        
        return K.reshape(x, (-1,input_shape[1],self.output_dim[1]))

    def compute_output_shape(self, input_shape):
        print('output' ,input_shape)
        return (input_shape[0], self.output_dim)

In [124]:
model2 = Sequential()
model2.add(MyReshape((look_back,1),input_shape=(look_back,)))
#model2.add( ReformatLayer( look_back,input_shape=(1,)))
#model2.add(Reshape((look_back,1),input_shape=(look_back,)))
#model2.add(Conv1D(120,5 , activation='relu'))
model2.add(Conv1D(120,5 , activation='relu'))
model2.add(Flatten())
model2.add(Dense(Ysize, activation='softmax'))


model2.summary()

buikd (None, 20)


ValueError: Tensor-typed variable initializers must either be wrapped in an init_scope or callable (e.g., `tf.Variable(lambda : tf.truncated_normal([10, 40]))`) when building functions. Please file a feature request if this restriction inconveniences you.

In [116]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(trainX0, trainYO, epochs=7,validation_split=0.1)

Train on 3608 samples, validate on 401 samples
Epoch 1/7
3608/3608 [==============================] - 0s 78us/sample - loss: 2.1913 - accuracy: 0.3276 - val_loss: 0.7319 - val_accuracy: 1.0000
Epoch 2/7
3608/3608 [==============================] - 0s 36us/sample - loss: 1.7278 - accuracy: 0.4678 - val_loss: 0.4412 - val_accuracy: 1.0000
Epoch 3/7
3608/3608 [==============================] - 0s 38us/sample - loss: 1.5690 - accuracy: 0.5238 - val_loss: 0.3118 - val_accuracy: 1.0000
Epoch 4/7
3608/3608 [==============================] - 0s 67us/sample - loss: 1.4431 - accuracy: 0.5660 - val_loss: 0.2762 - val_accuracy: 1.0000
Epoch 5/7
3608/3608 [==============================] - 0s 67us/sample - loss: 1.3443 - accuracy: 0.6070 - val_loss: 0.2473 - val_accuracy: 1.0000
Epoch 6/7
3608/3608 [==============================] - 0s 67us/sample - loss: 1.2529 - accuracy: 0.6286 - val_loss: 0.1991 - val_accuracy: 1.0000
Epoch 7/7
3608/3608 [==============================] - 0s 67us/sample - loss:

In [26]:
model = Sequential()
model.add(Conv1D(120,5 , activation='relu', input_shape=(look_back,1)))
model.add(MaxPooling1D(1))
model.add(Conv1D(20, 2, activation='relu'))
#model.add(MaxPooling1D(1))
model.add(Dropout(0.01))
#model.add(Dense(128))
#model.add(Conv1D(128,1 ))

model.add(Flatten())
#model.add(Conv1D(128,1 ))
#model.add(MaxPooling1D())
#model.add(Conv1D(128, 1, activation='relu'))
#model.add(GlobalAveragePooling1D())
#model.add(Dropout(0.3))
model.add(Dense(Ysize, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 16, 120)           720       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 16, 120)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 15, 20)            4820      
_________________________________________________________________
dropout (Dropout)            (None, 15, 20)            0         
_________________________________________________________________
flatten (Flatten)            (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                4816      
Total params: 10,356
Trainable params: 10,356
Non-trainable params: 0
____________________________________________________

In [28]:
#model.compile(loss='categorical_crossentropy',
#              optimizer='adam',
#              metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainYO, epochs=70,validation_split=0.1)

Train on 3608 samples, validate on 401 samples
Epoch 1/70
3608/3608 [==============================] - 0s 112us/sample - loss: 2.3075 - accuracy: 0.2971 - val_loss: 0.7321 - val_accuracy: 1.0000
Epoch 2/70
3608/3608 [==============================] - 0s 52us/sample - loss: 1.7821 - accuracy: 0.4498 - val_loss: 0.3874 - val_accuracy: 1.0000
Epoch 3/70
3608/3608 [==============================] - 0s 51us/sample - loss: 1.5577 - accuracy: 0.5288 - val_loss: 0.3246 - val_accuracy: 1.0000
Epoch 4/70
3608/3608 [==============================] - 0s 50us/sample - loss: 1.4009 - accuracy: 0.5757 - val_loss: 0.3632 - val_accuracy: 1.0000
Epoch 5/70
3608/3608 [==============================] - 0s 52us/sample - loss: 1.2858 - accuracy: 0.6247 - val_loss: 0.1938 - val_accuracy: 1.0000
Epoch 6/70
3608/3608 [==============================] - 0s 52us/sample - loss: 1.1744 - accuracy: 0.6574 - val_loss: 0.1461 - val_accuracy: 1.0000
Epoch 7/70
3608/3608 [==============================] - 0s 52us/sample

3608/3608 [==============================] - 0s 76us/sample - loss: 0.1692 - accuracy: 0.9440 - val_loss: 0.0221 - val_accuracy: 1.0000
Epoch 57/70
3608/3608 [==============================] - 0s 75us/sample - loss: 0.1714 - accuracy: 0.9432 - val_loss: 0.0094 - val_accuracy: 1.0000
Epoch 58/70
3608/3608 [==============================] - 0s 75us/sample - loss: 0.1626 - accuracy: 0.9471 - val_loss: 0.0066 - val_accuracy: 1.0000
Epoch 59/70
3608/3608 [==============================] - 0s 76us/sample - loss: 0.1594 - accuracy: 0.9446 - val_loss: 0.0048 - val_accuracy: 1.0000
Epoch 60/70
3608/3608 [==============================] - 0s 74us/sample - loss: 0.1563 - accuracy: 0.9473 - val_loss: 0.0071 - val_accuracy: 1.0000
Epoch 61/70
3608/3608 [==============================] - 0s 76us/sample - loss: 0.1489 - accuracy: 0.9507 - val_loss: 0.0081 - val_accuracy: 1.0000
Epoch 62/70
3608/3608 [==============================] - 0s 76us/sample - loss: 0.1505 - accuracy: 0.9529 - val_loss: 0.0030

In [11]:
nn=model.predict(trainX[1:30])
print('ll',nn)
nn2=numpy.argmax(nn,1)
print(nn2)
print(trainY[1:30])

ll [[4.64789046e-05 1.27639040e-01 1.01697037e-03 7.79515680e-14
  3.54250398e-04 9.94177535e-04 2.15455401e-03 2.10913640e-06
  1.54945269e-07 7.37730659e-07 2.88686570e-05 5.56765609e-17
  1.60258524e-02 8.51084054e-01 3.92470356e-05 6.13519282e-04]
 [4.99964245e-02 3.78726693e-14 1.57194881e-06 9.21752317e-21
  3.63504732e-05 1.03601838e-08 1.01050839e-03 1.55618356e-03
  3.21657353e-06 9.44499373e-01 6.39095379e-05 2.87898911e-05
  6.57187002e-07 2.36565247e-03 3.87233342e-07 4.37006587e-04]
 [7.75541703e-04 8.57356135e-05 9.98414636e-01 4.57845408e-06
  3.61598999e-04 3.29468778e-04 1.60045802e-05 2.89503149e-23
  4.39330461e-19 4.79616954e-12 1.34675054e-24 5.68114933e-09
  3.20404638e-06 4.67437440e-06 6.65284782e-18 4.65360699e-06]
 [5.97485632e-06 1.43172871e-03 9.72558837e-03 2.32251679e-07
  1.72865798e-03 2.22913150e-05 5.26649784e-03 1.22006040e-03
  4.15046024e-08 2.02487674e-04 4.32640547e-03 2.56399856e-19
  9.63680804e-01 1.23628890e-02 1.21503726e-05 1.41252376e-05]
 